<img src="valorumdata.svg" height="100px" width="200px" style="float: right;"/>

# Pandas: Basic functionality
---

**Prerequisites**

- [Pandas Intro](./intro.ipynb)

**Outcomes**

- Introduced to `datetime`
- Use built-in reduction functions and be able to create your own and apply it using `agg`
- Use `applymap` and `apply` to do "not reductions"
- Be able to select subsets of the DataFrame using boolean selection

**Data**

- US regional unemployment data from Bureau of Labor Statistics

---
    


In this notebook, we will use the same data as we used in [Pandas Intro](intro.ipynb), but will use monthly data rather than data from every other year

In [1]:
import pandas as pd
import valorum

%matplotlib inline

In [2]:
pd.__version__

'0.20.3'

In [3]:
## Load up the data
unemp = valorum.load("state_employment")

states = [
    "Arizona", "California", "Florida", "Illinois", 
    "Michigan", "New York", "Texas"
]

# Don't worry about what happens here for now
unemp = (unemp.reset_index()
              .pivot(index="Date", columns="state", values="UnemploymentRate")[states])

In [4]:
unemp.head()

state       Arizona  California  Florida  Illinois  Michigan  New York  Texas
Date                                                                         
2000-01-01      4.1         5.0      3.7       4.2       3.3       4.7    4.6
2000-02-01      4.0         5.0      3.7       4.2       3.2       4.6    4.6
2000-03-01      4.0         5.0      3.7       4.3       3.2       4.6    4.5
2000-04-01      4.0         5.1      3.7       4.3       3.3       4.6    4.4
2000-05-01      4.0         5.1      3.7       4.3       3.5       4.6    4.3

**Exercise**: Looking at the displayed DataFrame above, can you identify the index? The columns?

## Dates in Pandas

You might have noticed that our index now has a nice format for the dates (`YYYY-MM-DD`) rather than just a year

This is because the `dtype` of the outer-most index is a `datetime`.

In [5]:
unemp.index

DatetimeIndex(['2000-01-01', '2000-02-01', '2000-03-01', '2000-04-01',
               '2000-05-01', '2000-06-01', '2000-07-01', '2000-08-01',
               '2000-09-01', '2000-10-01',
               ...
               '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
               '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01',
               '2017-09-01', '2017-10-01'],
              dtype='datetime64[ns]', name='Date', length=214, freq=None)

We can index into a DataFrame with a `DatetimeIndex` using string representations of dates

For example

In [6]:
unemp.loc["01/01/2000", :]

state
Arizona       4.1
California    5.0
Florida       3.7
Illinois      4.2
Michigan      3.3
New York      4.7
Texas         4.6
Name: 2000-01-01 00:00:00, dtype: float64

In [7]:
unemp.loc["01/01/2000":"06/01/2000", :]

state       Arizona  California  Florida  Illinois  Michigan  New York  Texas
Date                                                                         
2000-01-01      4.1         5.0      3.7       4.2       3.3       4.7    4.6
2000-02-01      4.0         5.0      3.7       4.2       3.2       4.6    4.6
2000-03-01      4.0         5.0      3.7       4.3       3.2       4.6    4.5
2000-04-01      4.0         5.1      3.7       4.3       3.3       4.6    4.4
2000-05-01      4.0         5.1      3.7       4.3       3.5       4.6    4.3
2000-06-01      4.0         5.0      3.8       4.3       3.6       4.6    4.3

The `datetime` object in Python will also facilitate other interactions with dates and time based information

For example, if we wanted to get the day of the week, month, year, or other information then we can access it as shown below

In [8]:
unemp.index.year

Int64Index([2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
            ...
            2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017, 2017],
           dtype='int64', name='Date', length=214)

In [9]:
unemp.index.month

Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
            ...
             1,  2,  3,  4,  5,  6,  7,  8,  9, 10],
           dtype='int64', name='Date', length=214)

In [10]:
unemp.index.dayofweek

Int64Index([5, 1, 2, 5, 0, 3, 5, 1, 4, 6,
            ...
            6, 2, 2, 5, 0, 3, 5, 1, 4, 6],
           dtype='int64', name='Date', length=214)

In [11]:
unemp.index.dayofyear

Int64Index([  1,  32,  61,  92, 122, 153, 183, 214, 245, 275,
            ...
              1,  32,  60,  91, 121, 152, 182, 213, 244, 274],
           dtype='int64', name='Date', length=214)

We will discuss `datetime` objects in greater detail in an upcoming notebook

## DataFrame reductions

You can think of a _reduction_ as an operation that turns something with multiple values into a single value (See [Wikipedia](https://en.wikipedia.org/wiki/Reduction_Operator) for a more formal definition)

For example, computing the mean of three numbers (for example `[0, 1, 2]`), returns a single number (1)

We will use reductions extensively as we analyze and manipulate our data


### Built-in reductions

Pandas already has some of the most frequently used reductions built-in

For example:

* Mean
* Variance
* Standard deviation
* Minimum
* Median
* Maximum
* etc...

When looking for standard operations, using some "tab completion" goes a long ways.

In [12]:
unemp.mean()

state
Arizona       6.299533
California    7.318224
Florida       6.065421
Illinois      6.841589
Michigan      7.500935
New York      6.109346
Texas         5.707009
dtype: float64

As seen above, reduction reduces by columns by default... However, by using `axis` keyword argument, you can do reductions by row as well

In [13]:
unemp.var(axis=1).head()

Date
2000-01-01    0.352381
2000-02-01    0.374762
2000-03-01    0.364762
2000-04-01    0.353333
2000-05-01    0.294762
dtype: float64

### Writing your own reduction

In addition to allowing individuals to use the most frequently used reductions, Pandas has made it easy to write your own reductions.

We can do this in two steps:

1. Write a function that takes a single `Series` as an input and output a single value
2. Call the `agg` method with that function as an argument

For example, below we will classify states as "low unemployment" or "high unemployment" based on whether their mean unemployment level is above or below 6.5.


In [14]:
#
# Step 1: We write the (reduction) function that we'd like to use
#
def high_or_low(s):
    """
    This function takes a Pandas Series object and returns high
    if the mean is above 6.5 and low if the mean is below 6.5
    """
    if s.mean() < 6.5:
        out = "Low"
    else:
        out = "High"

    return out


In [15]:
#
# Step 2: Apply it via the agg method
#
unemp.agg(high_or_low)

state
Arizona        Low
California    High
Florida        Low
Illinois      High
Michigan      High
New York       Low
Texas          Low
dtype: object

In [16]:
# How does this differ from unemp.agg(high_or_low)?
unemp.agg(high_or_low, axis=1).head()  

Date
2000-01-01    Low
2000-02-01    Low
2000-03-01    Low
2000-04-01    Low
2000-05-01    Low
dtype: object

Notice that `agg` can also accept multiple functions at once

In [17]:
unemp.agg([min, max, high_or_low])

Arizona California Florida Illinois Michigan New York Texas
min             3.7        4.7     3.1      4.2      3.2      4.2   3.9
max            11.2       12.2    11.2     11.2     14.9      8.9   8.4
high_or_low     Low       High     Low     High     High      Low   Low

**Exercises**:


- At each date what was the minimum unemployment rate across all states in our sample?
- What was the median unemployment rate in each state?
- What was the maximum unemployment rate across the states in our sample? What state did it happen in? In what month/year was this achieved?
    - Hint 1: What type is returned by a reduction.
    - Hint 2: Read documentation for the method `idxmax`.
- Classify each state as high or low volatility based on whether the variance of their unemployment is above or below 4.


## Not reductions

Many analytical operations do not necessarily involve a reduction.

The output of a function applied to a Series might need to be a new Series

For example, you might want to compute the percentage change in unemployment between each month, or compute the cumulative sum of its elements.

### Built-in "not reductions"

Pandas has also provided some of the more common not reduction functions that you might need, including:

* Cumulative sum/max/min/product
* Difference
* Elementwise addition/division/multiplication/subtraction
* Percent change
* Number of occurrences of each distinct value (`value_counts`)

Again, tab completion is helpful when trying to find these functions

In [18]:
unemp.head()

state       Arizona  California  Florida  Illinois  Michigan  New York  Texas
Date                                                                         
2000-01-01      4.1         5.0      3.7       4.2       3.3       4.7    4.6
2000-02-01      4.0         5.0      3.7       4.2       3.2       4.6    4.6
2000-03-01      4.0         5.0      3.7       4.3       3.2       4.6    4.5
2000-04-01      4.0         5.1      3.7       4.3       3.3       4.6    4.4
2000-05-01      4.0         5.1      3.7       4.3       3.5       4.6    4.3

In [19]:
unemp.pct_change().head()

state       Arizona  California  Florida  Illinois  Michigan  New York  \
Date                                                                     
2000-01-01      NaN         NaN      NaN       NaN       NaN       NaN   
2000-02-01 -0.02439        0.00      0.0   0.00000 -0.030303 -0.021277   
2000-03-01  0.00000        0.00      0.0   0.02381  0.000000  0.000000   
2000-04-01  0.00000        0.02      0.0   0.00000  0.031250  0.000000   
2000-05-01  0.00000        0.00      0.0   0.00000  0.060606  0.000000   

state          Texas  
Date                  
2000-01-01       NaN  
2000-02-01  0.000000  
2000-03-01 -0.021739  
2000-04-01 -0.022222  
2000-05-01 -0.022727

In [20]:
unemp.diff().head()

state       Arizona  California  Florida  Illinois  Michigan  New York  Texas
Date                                                                         
2000-01-01      NaN         NaN      NaN       NaN       NaN       NaN    NaN
2000-02-01     -0.1         0.0      0.0       0.0      -0.1      -0.1    0.0
2000-03-01      0.0         0.0      0.0       0.1       0.0       0.0   -0.1
2000-04-01      0.0         0.1      0.0       0.0       0.1       0.0   -0.1
2000-05-01      0.0         0.0      0.0       0.0       0.2       0.0   -0.1

### Writing your own not reduction

In addition to allowing individuals to use the most frequently used reductions, Pandas has made it easy to write your own reductions.

This can be done in three steps:

1. Write a function that takes a Series and outputs a new Series
2. Determine whether the operation is more suitable for `apply` or `applymap`
3. Call the function using either `apply` or `applymap`

How do you know when to use each of these two methods?

* `applymap` is used to change one scalar (single value) into another. For example, taking the absolute value or adding a constant number to each element of a DataFrame.
* `apply` takes a Series and turns it into either a scalar or another Series. For example, computing the percent change or taking the cumulative sum -- These are operations that `applymap` could not perform because they depend on other elements of the Series. 

As an example, we will standardize our unemployment data to have mean 0 and standard deviation 1. We can use this (and a reduction) to determine at which point in time the unemployment rate is most different from "normal times" for each state.

In [21]:
unemp.head()

state       Arizona  California  Florida  Illinois  Michigan  New York  Texas
Date                                                                         
2000-01-01      4.1         5.0      3.7       4.2       3.3       4.7    4.6
2000-02-01      4.0         5.0      3.7       4.2       3.2       4.6    4.6
2000-03-01      4.0         5.0      3.7       4.3       3.2       4.6    4.5
2000-04-01      4.0         5.1      3.7       4.3       3.3       4.6    4.4
2000-05-01      4.0         5.1      3.7       4.3       3.5       4.6    4.3

In [22]:
def standardize_data(x):
    """
    Changes the data in a Series to become mean 0 with standard deviation 1
    """
    mu = x.mean()
    std = x.std()

    return (x - mu)/std


In [23]:
std_unemp = unemp.apply(standardize_data)
std_unemp.head()

state        Arizona  California   Florida  Illinois  Michigan  New York  \
Date                                                                       
2000-01-01 -1.052089   -0.953231 -0.999747 -1.361819 -1.607316 -0.933649   
2000-02-01 -1.099921   -0.953231 -0.999747 -1.361819 -1.645577 -0.999896   
2000-03-01 -1.099921   -0.953231 -0.999747 -1.310266 -1.645577 -0.999896   
2000-04-01 -1.099921   -0.912112 -0.999747 -1.310266 -1.607316 -0.999896   
2000-05-01 -1.099921   -0.912112 -0.999747 -1.310266 -1.530794 -0.999896   

state          Texas  
Date                  
2000-01-01 -0.876326  
2000-02-01 -0.876326  
2000-03-01 -0.955488  
2000-04-01 -1.034649  
2000-05-01 -1.113811

In [24]:
# Takes the absolute value of all elements of a function
abs_std_unemp = std_unemp.abs()

abs_std_unemp.head()

state        Arizona  California   Florida  Illinois  Michigan  New York  \
Date                                                                       
2000-01-01  1.052089    0.953231  0.999747  1.361819  1.607316  0.933649   
2000-02-01  1.099921    0.953231  0.999747  1.361819  1.645577  0.999896   
2000-03-01  1.099921    0.953231  0.999747  1.310266  1.645577  0.999896   
2000-04-01  1.099921    0.912112  0.999747  1.310266  1.607316  0.999896   
2000-05-01  1.099921    0.912112  0.999747  1.310266  1.530794  0.999896   

state          Texas  
Date                  
2000-01-01  0.876326  
2000-02-01  0.876326  
2000-03-01  0.955488  
2000-04-01  1.034649  
2000-05-01  1.113811

In [25]:
abs_std_unemp.agg(lambda x: x.idxmax())

state
Arizona      2009-11-01
California   2010-02-01
Florida      2009-11-01
Illinois     2009-12-01
Michigan     2009-06-01
New York     2009-10-01
Texas        2009-08-01
dtype: datetime64[ns]

**Exercises**:

- Imagine we wanted to determine whether unemployment was high (> 6.5), medium (4.5 < x <= 6.5), or low (<= 4.5) for each state and each month. Would you use `agg`, `apply`, or `applymap`? Discuss with your neighbor.
- Write a function and use the appropriate method to create a new DataFrame that implements the computation described above.
- Apply your function to the DataFrame and save it in a variable called `unemp_bins`
- (Challenging) This exercise has multiple parts:
    1. Use another "not reduction" on `unemp_bins` to count how many times each state had each of the three classifications.
        - Hint 1: Will you need to `apply` or `applymap` the not reduction function?
        - Hint 2: Try googling "pandas count unique value" or something similar to find the right not reduction.
    2. Construct a horizontal bar chart of the number of occurrences of each level with one bar per state.
 
- (Challenging) Repeat the previous step, but count how many states had each classification in each month. Which month had the most states with high unemployment? What about medium and low?


## Boolean Selection

We have seen how we can select subsets of data by referring to the index or column names

However, often we want to do our selection based on conditions met by the data itself and don't know which observations satisfy these conditions

Some examples are

* Restrict analysis to all individuals older than 18
* Look at data that corresponds to particular time periods
* Analyze only data that corresponds to a recession
* Obtain data for a specific product or customer ID

We will be able to do this by using a Series of boolean values to index into a Series or DataFrame

Let's look at some examples

In [26]:
unemp_small = unemp.head()  # Create smaller data so we can see what's happening
unemp_small

state       Arizona  California  Florida  Illinois  Michigan  New York  Texas
Date                                                                         
2000-01-01      4.1         5.0      3.7       4.2       3.3       4.7    4.6
2000-02-01      4.0         5.0      3.7       4.2       3.2       4.6    4.6
2000-03-01      4.0         5.0      3.7       4.3       3.2       4.6    4.5
2000-04-01      4.0         5.1      3.7       4.3       3.3       4.6    4.4
2000-05-01      4.0         5.1      3.7       4.3       3.5       4.6    4.3

In [27]:
unemp_small.loc[[True, True, True, False, False]]

state       Arizona  California  Florida  Illinois  Michigan  New York  Texas
Date                                                                         
2000-01-01      4.1         5.0      3.7       4.2       3.3       4.7    4.6
2000-02-01      4.0         5.0      3.7       4.2       3.2       4.6    4.6
2000-03-01      4.0         5.0      3.7       4.3       3.2       4.6    4.5

In [28]:
unemp_small.loc[[True, False, True, False, True], :]

state       Arizona  California  Florida  Illinois  Michigan  New York  Texas
Date                                                                         
2000-01-01      4.1         5.0      3.7       4.2       3.3       4.7    4.6
2000-03-01      4.0         5.0      3.7       4.3       3.2       4.6    4.5
2000-05-01      4.0         5.1      3.7       4.3       3.5       4.6    4.3

In [29]:
unemp_small.loc[[True, True, True, False, False], [True, False, False, False, False, True, True]]

state       Arizona  New York  Texas
Date                                
2000-01-01      4.1       4.7    4.6
2000-02-01      4.0       4.6    4.6
2000-03-01      4.0       4.6    4.5

### Creating Boolean DataFrames/Series

We can use conditional statements to create DataFrames/Series with boolean values and later we will use these to select particular subsets of information.

In [30]:
def high_med_low(x):
    if x < 4.5:
        return "Low"
    elif x < 6.5:
        return "Medium"
    else:
        return "High"

unemp_class = unemp.applymap(high_med_low)
unemp_class.head()

state      Arizona California Florida Illinois Michigan New York   Texas
Date                                                                    
2000-01-01     Low     Medium     Low      Low      Low   Medium  Medium
2000-02-01     Low     Medium     Low      Low      Low   Medium  Medium
2000-03-01     Low     Medium     Low      Low      Low   Medium  Medium
2000-04-01     Low     Medium     Low      Low      Low   Medium     Low
2000-05-01     Low     Medium     Low      Low      Low   Medium     Low

#### Conditional statements

With conditional statements (like we learned in the [boolean notebook](../pyfun/booleans.ipynb)), we check where certain conditions are true

In [31]:
unemp_small

state       Arizona  California  Florida  Illinois  Michigan  New York  Texas
Date                                                                         
2000-01-01      4.1         5.0      3.7       4.2       3.3       4.7    4.6
2000-02-01      4.0         5.0      3.7       4.2       3.2       4.6    4.6
2000-03-01      4.0         5.0      3.7       4.3       3.2       4.6    4.5
2000-04-01      4.0         5.1      3.7       4.3       3.3       4.6    4.4
2000-05-01      4.0         5.1      3.7       4.3       3.5       4.6    4.3

In [32]:
unemp_small["Texas"] < 4.5

Date
2000-01-01    False
2000-02-01    False
2000-03-01    False
2000-04-01     True
2000-05-01     True
Freq: MS, Name: Texas, dtype: bool

In [33]:
unemp_small["New York"] > unemp_small["Texas"]

Date
2000-01-01     True
2000-02-01    False
2000-03-01     True
2000-04-01     True
2000-05-01     True
Freq: MS, dtype: bool

In [34]:
(unemp_class["New York"] == "Low").head()

Date
2000-01-01    False
2000-02-01    False
2000-03-01    False
2000-04-01    False
2000-05-01    False
Freq: MS, Name: New York, dtype: bool

#### Multiple conditions

We could chain conditional statements in the [boolean notebook](../pyfun/booleans/ipynb) using the words `and` and `or`

Recall

* `True and False -> False`
* `True and True -> True`
* `False and False -> False`
* `True or False -> True`
* `True or True -> True`
* `False or False -> False`

When we would like to check multiple conditions in Pandas, we use different notation `&` in place of `and` and `|` in place of `or` (note that we need to have parenthesis surrounding each condition)

In [35]:
((unemp_class["Texas"] == "Low") | (unemp_class["Texas"] == "Medium")).head()

Date
2000-01-01    True
2000-02-01    True
2000-03-01    True
2000-04-01    True
2000-05-01    True
Freq: MS, Name: Texas, dtype: bool

In [36]:
(unemp_small["Texas"] < 4.7) & (unemp_small["New York"] < 4.7)

Date
2000-01-01    False
2000-02-01     True
2000-03-01     True
2000-04-01     True
2000-05-01     True
Freq: MS, dtype: bool

#### `isin`

Sometimes we will want to check whether something takes one of multiple values

We could do this by writing `(df["x"] == val_1) | (df["x"] == val_2)` (like we did above), but this could make your code quite long

Pandas has an easier way to generate booleans of this form

In [37]:
unemp_class.head()

state      Arizona California Florida Illinois Michigan New York   Texas
Date                                                                    
2000-01-01     Low     Medium     Low      Low      Low   Medium  Medium
2000-02-01     Low     Medium     Low      Low      Low   Medium  Medium
2000-03-01     Low     Medium     Low      Low      Low   Medium  Medium
2000-04-01     Low     Medium     Low      Low      Low   Medium     Low
2000-05-01     Low     Medium     Low      Low      Low   Medium     Low

In [38]:
unemp_class["Texas"].isin(["Low", "High"]).head()

Date
2000-01-01    False
2000-02-01    False
2000-03-01    False
2000-04-01     True
2000-05-01     True
Freq: MS, Name: Texas, dtype: bool

In [39]:
unemp_class["New York"].isin(["Low", "High"]).head()

Date
2000-01-01    False
2000-02-01    False
2000-03-01    False
2000-04-01    False
2000-05-01    False
Freq: MS, Name: New York, dtype: bool

### Using the boolean Series

The previous section demonstrated how to create Series of boolean values

Recall that this is useful because we can use these Series to index into our data

In [40]:
unemp_class.loc[unemp_class["Texas"] == "High"].head()

state      Arizona California Florida Illinois Michigan New York Texas
Date                                                                  
2002-10-01  Medium       High  Medium     High   Medium   Medium  High
2002-11-01  Medium       High  Medium     High   Medium   Medium  High
2002-12-01  Medium       High  Medium     High   Medium   Medium  High
2003-01-01  Medium       High  Medium     High     High   Medium  High
2003-02-01  Medium       High  Medium     High     High   Medium  High

In [41]:
unemp_class.loc[unemp_class["New York"] == "Low"].head()

state      Arizona California Florida Illinois Michigan New York Texas
Date                                                                  
2000-10-01     Low     Medium     Low      Low      Low      Low   Low
2000-11-01     Low     Medium     Low   Medium      Low      Low   Low
2000-12-01     Low     Medium     Low   Medium      Low      Low   Low
2001-01-01     Low     Medium     Low   Medium      Low      Low   Low
2001-02-01     Low     Medium     Low   Medium   Medium      Low   Low

In [42]:
# Do you recall what any does?
unemp_class.loc[unemp_class.isin(["High"]).any(axis=1)].head()

state      Arizona California Florida Illinois Michigan New York   Texas
Date                                                                    
2002-01-01  Medium       High  Medium   Medium   Medium   Medium  Medium
2002-02-01  Medium       High  Medium     High   Medium   Medium  Medium
2002-03-01  Medium       High  Medium     High   Medium   Medium  Medium
2002-04-01  Medium       High  Medium     High   Medium   Medium  Medium
2002-05-01  Medium       High  Medium     High   Medium   Medium  Medium

In [43]:
# Do you recall what all does?
unemp_class.loc[unemp_class.isin(["High"]).all(axis=1)].head()

state      Arizona California Florida Illinois Michigan New York Texas
Date                                                                  
2009-03-01    High       High    High     High     High     High  High
2009-04-01    High       High    High     High     High     High  High
2009-05-01    High       High    High     High     High     High  High
2009-06-01    High       High    High     High     High     High  High
2009-07-01    High       High    High     High     High     High  High

**Exercises**:

- For a single state of your choice, determine what the mean unemployment is during "Low", "Medium", and "High" unemployment times.
    - This will require you to use both the `unemp_class` and `unemp` DataFrames. 
    - Discuss with your neighbor how you would do this for all the states in our sample... We will soon learn tools that will _greatly_ simplify this type of operation.
- Which states in our sample performs the best during "bad times?" To determine this, compute the mean unemployment for each state for only months in which the mean unemployment rate in our sample is greater than 7 
